In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelBinarizer, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# train = pd.read_csv('./data/rawdata/application_train.csv.zip',compression='zip')
# test = pd.read_csv('./data/rawdata/application_test.csv.zip',compression='zip')

pre = pd.read_csv('./data/rawdata/previous_application.csv.zip',compression='zip')
bureau = pd.read_csv('./data/rawdata/bureau.csv.zip',compression='zip')
credit = pd.read_csv('./data/rawdata/credit_card_balance.csv.zip',compression='zip')